# Evaluación python 1
## solo para ejercitar (el que dió la profe)

## PROBLEMA 01
En farmacocinética, la concentración plasmática de un fármaco se estudia para garantizar que el paciente tenga niveles terapéuticos seguros. Supongamos que la concentración del fármaco en el torrente sanguíneo C(t) (en mg/L) sigue el modelo:
$$ \frac{dC}{dt}=D - \lambda C \quad ; \quad C(0)=C$$
donde
* D>0 es la tasa de administración del fármaco, expresada en mg/L*h, considerada constante debido al uso de una bomba de infusión intravenosa
* $\lambda$ > 0 es la constante de eliminación metabólica y renal del organismo, expresada en $h^{-1}$
* $C_0$ es la concentración inicial del fármaco en plasma (mg/L)
* La variable t representa el tiempo medido en horas (h).
Este modelo lineal simplificado corresponde a una situación de infusión intravenosa continua, donde la entrada del fármaco es constante y la eliminación es proporcional a la concentradio.

**(a)** Mediante un análisis cualitativo de la ecuación diferencial (sin resolverla). Determine para qué valores de la concentración inicial $C_0$ la solución C(t) será:
* creciente
* decreciente
Interprete los resultados en térmicos clínicos: ¿qué significa que la concentración del fármaco aumente o disminuya en la sangre tras iniciar la infusión?

### RESPUESTA (a)
ya que se busca los valores de C para que sea creciente o decreciente significa que hay que buscar los valores en los que $ \frac{dC}{dt}$ sea 0 (punto de equilibrio o estacionario). Con la derivada > 0 significa que es creciente, con < 0 es decreciente, = 0 no cambia la concentracion. entonces hay que despejar C de la siguiente ecuación $ 0= D-\lambda C$ así dando como resultado $ C = D/\lambda$. Con esto y el contexto clinico, C no puede ser menor a 0 (valores negativos), así en el tramo de $0 <= C < D/\lambda$ la función será creciente. Y para todo valor $ C > D/\lambda$ será decreciente.
Esto en el contexto del fármaco, si es creciente significa que el fármaco se acumula en el cuerpo (entra más del que sale) en el caso de decreciente pasa lo contrario (sale más del que entra)

**(b)** Calcule la concentración de equilibrio y explique por qué este valor corresponde a la concentración plasmática en estado estacionario. Discuta la importancia de alcanzar dicho estado en tratamientos de larga duración y los riesgos de desviarse de este equilibrio (toxicidad o ineficacia).

### RESPUESTA (b)

como se calculó anteriormente, la concentración de equilibrio es $C = D/\lambda$. Es importante alcanzar este estado ya que como se mencionó en el apartado a, al estar fuera del punto de equilibrio hay 2 casos, que se acumule el farmaco en el cuerpo así causando toxicidad o otros problemas de salud, o que salga el farmaco del cuerpo así causando ineficacia por lo que es como si nunca se medicara al paciente.

## PROBLEMA 02
En un sistema de ventilación mecánica, un pulmón artificial de volumen máximo $V$ (litros) recibe mezcla gaseosa desde un compresor $A$ a razón de $a$ L/min confracción inspirada en oxígeno $ c \in (0, 1) $ . Simultáneamente, el paciente exhala a razón de $b$ L/min hacía el exterior $B$. Sea $x(t)$ la cantidad de oxígeno almacenado en el pulmón artificial (en litros de $O_2$) y $V(T) = V - (b-a)t$ el volumen de gas dentro del pulmón (suponiendo $b > a$).
Además del arrastre por el flujo de salida, existe un consumo metabólico proporcional a la fracción de oxígeno en el pulmón, modelado con coeficiente $k > 0$ (min$^{-1}$). El balance de masa de $O_2$ queda:
$$ \frac{dx}{dt}=ca - \frac{bx}{V(t)}- \frac{kx}{V(t)} \quad ; \quad V(t) = V - (b-a)t$$
Aquí t está en minutos.
Use los parámetros: $V=600, a=5, b=6, c=0.21, k=2$ y condición inicial $x(0) = 120$

**(a)** Estime $x(30)$ con **Euler** y **RK4** usando $h=0,1$.

In [35]:
#importar todo
import matplotlib.pyplot as plt
import numpy as np
import numpy as np
import sympy as sp
from sympy import *
sp.init_printing(use_latex='mathjax')

#definir Euler
def Euler(f,x0,xn,y0,n):
    X = np.linspace(x0,xn,n+1)         
    Y = np.linspace(x0,xn,n+1)                 
    Y[0] = y0
    h=(xn-x0)/n
    for i in range(n):
        Y[i+1] = Y[i] + h*f(X[i],Y[i])
    return Y

#definir RK4
def RK4(f, x0, xn, y0, n):
    X = np.linspace(x0,xn,n+1)
    Y = np.linspace(x0,xn,n+1)                 
    Y[0] = y0        
    h = (xn-x0)/n
    for i in range(n):
        K1=f(X[i],Y[i]) 
        K2=f(X[i]+h/2,Y[i]+(h/2)*K1) 
        K3=f(X[i]+h/2,Y[i]+(h/2)*K2) 
        K4=f(X[i]+h,Y[i]+h*K3)
        Y[i+1] = Y[i] +(h/6)*(K1+2*K2+2*K3+K4)        
    return Y
    
#definir la funcion del ejercicio
def f(t,x): return 0.21*5 - (6*x)/(600-t) - (2*x)/(600-t)

#para h=0,1 significa que n debe ser 300
V_aprox_euler = Euler(f,0,30,120,300)[-1]
V_aprox_rk4 = RK4(f,0,30,120,300)[-1]
print("aproximación con Euler:",V_aprox_euler)
print("aproximación con RK4:",V_aprox_RK4 )

aproximación con Euler: 105.39772099405836
aproximación con RK4: 105.40261293867214


**(b)** Obtenga la solución exca en PYTHON (incluya el factor integrante explícito) y calcule el error absoluto de cada método respecto de la solución exacta y discuta si es aceptable para control de soporte ventilatorio.

In [29]:
#La forma de la edo es lineal, dy/dx+P(X)*y=Q(x)
#para trabajarlo con factor integrante explicito hay que reescribir la ecuación, así quedando
#dx/dt+((-k-b)/(600-t))*x = c*a
#así siendo P(x) = (-k-b)/(600-t) = -8/(600-t)
#u = e^(integral de P(x)*dx)
t = sp.Symbol("t", real=True)
x = sp.Function("x")

integral = sp.integrate(-8/(600-t),t)
print("resultado de la integral:",integral)
u = sp.exp(integral)
print("el factor integrante es:", u)

resultado de la integral: 8*log(t - 600)
el factor integrante es: (t - 600)**8


In [32]:
#ya mostré el factor integrante, ahora calculo con el dsolve la solucion como el 
#resto de ejercicios que suelo hacer aquí
#remplazo 0.21 = sp.rational(21,100)
edo = sp.Rational(21,100)*5- (8*x(t))/(600-t) #la edo del enunciado
solucion = sp.dsolve(sp.diff(x(t),t) - edo, ics={x(0):120})
solucion

                  8                     7                    6               5
                 t                     t                  7⋅t             7⋅t 
x(t) = ───────────────────── - ────────────────── + ─────────────── - ────────
       559872000000000000000   116640000000000000   388800000000000   32400000

             4         3       2             
          7⋅t       7⋅t     7⋅t    11⋅t      
──── + ───────── - ────── + ──── - ──── + 120
0000   432000000   900000   3000    20       

In [33]:
#evaluar el x(30)
V_real = solucion.rhs.subs(t,30).evalf()
V_real
#rhs toma el lado derecho de una ecuacion de sp.Eq (en este caso de la solucion de la edo) 
#subs remplaza la variable que le des con el numero que quieras (t=30)
#y .evalf() calcula el valor numerico para esa funcion sustituida

105.402612938672

In [39]:
#calcular el error de cada metodo
error_euler = np.abs(V_real - V_aprox_euler)*100/V_real
error_rk4 = np.abs(V_real - V_aprox_rk4)*100/V_real
print("Error% del metodo de Euler:", error_euler,"%")
print("Error% del metodo RK4:", error_rk4,"%")
print("Ambos errores son tan bajos que se pueden considerar casi nulos debido al valor de h tan bajo, si se pueden usar en este contexto")

Error% del metodo de Euler: 0.00464119861655145 %
Error% del metodo RK4: 2.42684102169715e-13 %
Ambos errores son tan bajos que se pueden considerar casi nulos debido al valor de h tan bajo, si se pueden usar en este contexto
